# Flask improving and best practices

## Para migrar tu aplicación Flask a Docker y Nginx, puedes seguir los siguientes pasos:

1. Crear un archivo Dockerfile en la raíz de tu proyecto Flask. Este archivo contendrá las instrucciones para construir tu imagen de Docker. A continuación se muestra un ejemplo de Dockerfile:

``` Dockerfile
# seleccion de la imagen base
FROM python:3.9-slim-buster

# alteracion de la imagen base
# Install y configuracion base de nginx para funcionar solo en http
RUN apt-get update && \
    apt-get install -y nginx

# Remove the default Nginx configuration file
RUN rm /etc/nginx/sites-enabled/default

# Copy the custom Nginx configuration file
COPY nginx.conf /etc/nginx/sites-enabled/

# deploy app inside docker.
# nunca dejar credenciales dentro de un docker
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .

# inicio del webservice
CMD ["gunicorn", "--bind", "0.0.0.0:5000", "app:app"]
```

2. Crear un archivo de configuración para Nginx. Este archivo definirá cómo Nginx manejará las solicitudes HTTP que se envían a tu aplicación. A continuación se muestra un ejemplo de archivo de configuración de Nginx:

``` nginx.conf
server {
    listen 80;
    server_name example.com;

    location / {
        proxy_pass http://web:5000;
        proxy_set_header Host $host;
        proxy_set_header X-Real-IP $remote_addr;
    }
}
```

3. Crear un archivo docker-compose.yml en la raíz de tu proyecto. Este archivo contendrá la configuración para tu contenedor Docker. A continuación se muestra un ejemplo de docker-compose.yml:


``` docker-compose.yml
version: '3'

services:
  web:
    build: .
    ports:
      - "80:80"
    environment:
      - DEBUG=False
    depends_on:
      - db
  db:
    image: postgres
    environment:
      POSTGRES_USER: postgres
      POSTGRES_PASSWORD: example
```


